In [6]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms

torch.set_printoptions(edgeitems=2, linewidth=75)
data_path = '../data-unversioned/p1ch7/'

In [ ]:
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download= True,
                          transform=transforms.ToTensor())

In [10]:
img_t, _ = tensor_cifar10[99]
type(img_t)

torch.Tensor

In [11]:
img_t.shape, img_t.dtype

(torch.Size([3, 32, 32]), torch.float32)

In [12]:
img_t.min(), img_t.max()

(tensor(0.), tensor(1.))

In [14]:
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
imgs.shape

torch.Size([3, 32, 32, 50000])

In [15]:
imgs.view(3, -1).mean(dim=1)

tensor([0.4914, 0.4822, 0.4465])

In [16]:
imgs.view(3, -1).std(dim=1)

tensor([0.2470, 0.2435, 0.2616])

In [17]:
transformed_cifar10 = datasets.CIFAR10(
    data_path, train=True, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [18]:
transformed_cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

In [ ]:
from collections import OrderedDict

nn21_model = nn.Sequential(OrderedDict([
            ('hidden_linear_1', nn.Linear(input_layer,8)),
            ('hidden_activation_1', nn.Tanh()),
            ('hidden_linear_2', nn.Linear(8, 4)),
            ('hidden_activation_2', nn.Tanh()),
            ('output_linear', nn.Linear(4, 1))
]))
    